In [24]:
print("Pront allez!")

Pront allez!


In [25]:
from langchain_groq import ChatGroq

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [27]:
model = ChatGroq( model="qwen/qwen3-32b" )

In [28]:
model.invoke("What is the capital of France?").content

'<think>\nOkay, so the user is asking, "What is the capital of France?" Hmm, I need to make sure I answer this correctly. Let me start by recalling what I know about France. France is a country in Western Europe, right? I remember from geography class that the capital is Paris. But wait, maybe I should double-check to be sure. Sometimes countries have cities that are major cities but not capitals. For example, Lyon is a big city in France, but I don\'t think it\'s the capital. Marseille is another one, but again, not the capital. So Paris is the one. Let me think if there\'s any other city that might be confused with the capital. Oh, maybe Versailles? But Versailles is a city known for the Palace of Versailles, which is a famous landmark. But Versailles is part of the Île-de-France region, and Paris is the capital of that region. So the capital of France is definitely Paris. I should also consider if there\'s any other city that might be considered the capital in a different context, l

In [29]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

d:\Agentic AI\projects\DOCUMENT_PORTAL\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [31]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Embed a query
result = genai.embed_content(
    model="embedding-001",
    content="What is the capital of France?",
    task_type="retrieval_query",
)

embedding = result["embedding"]
print(f"Length: {len(embedding)}")
print("First 5 values:", embedding[:5])


Length: 768
First 5 values: [0.037376475, -0.0800528, 0.009497141, -0.0542599, 0.04019698]


1.DATA INGESTION

In [1]:
from langchain.document_loaders import PyPDFLoader

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
import os

In [6]:
file_path = os.path.join(os.getcwd(), "data", "sample.pdf")

In [4]:
os.getcwd()

'd:\\Agentic AI\\projects\\DOCUMENT_PORTAL\\notebook'

In [7]:
Documents = PyPDFLoader(file_path=file_path).load()

In [9]:
Documents[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\

In [10]:
len(Documents)

77

This is a experimental thing there is no diterministic way to dosplit the text

In [20]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

In [21]:
docs = text_splitter.split_documents(Documents)

In [22]:
len(docs)

1952

In [23]:
from langchain.vectorstores import FAISS

In [37]:
len(embedding_model.embed_documents(docs[0].page_content))

167

In [38]:
vector_store = FAISS.from_documents(docs, embedding_model)

This is the Retriever Process

Means from the vectorization we are going to fetch or rank the most appropriate result

In [41]:
relevant_doc = vector_store.similarity_search("llama2 finetuning benchmark experiment")

In [42]:
relevant_doc[0].page_content

'descriptions of the benchmarks and metrics can be found in Appendix A.4.7. When compared toLlama 1-7B,'

In [43]:
relevant_doc[1].page_content

'demographic attributes.\nWe compare the performance ofLlama 2with Llama 1(Touvron et al., 2023), Falcon (Almazrouei et al.,'

In [44]:
relevant_doc[2].page_content

'and PaLM-2-L.\nWe also analysed the potential data contamination and share the details in Section A.6.\nBenchmark (shots) GPT-3.5 GPT-4 PaLM PaLM-2-L Llama 2\nMMLU (5-shot) 70.0 86.4 69.3 78.3 68.9'

In [45]:
relevant_doc[3].page_content

'hyper-specialization (Scialom et al., 2020b), it is important before a newLlama 2-Chattuning iteration to'

In [46]:
retriever = vector_store.as_retriever()

In [48]:
retriever.invoke("llama2 finetuning benchmark experiment")

[Document(id='0e9ee084-686d-4a7e-b34d-567563f7bc54', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'd:\\Agentic AI\\projects\\DOCUMENT_PORTAL\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 21, 'page_label': '22'}, page_content='descriptions of the benchmarks and metrics can be found in Appendix A.4.7. When compared toLlama 1-7B,'),
 Document(id='dc33b5a1-66d3-4fe6-82f5-33be49ec2791', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.

In [49]:
prompt_template = """ Answer the question based on the context below. If you don't know the answer, just say that you 
don't know, don't try to make up an answer. 
Context: {context} 
Question: {question}
"""

In [50]:
from langchain.prompts import PromptTemplate

In [57]:
Prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
    )

In [58]:
Prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" Answer the question based on the context below. If you don't know the answer, just say that you \ndon't know, don't try to make up an answer. \nContext: {context} \nQuestion: {question}\n")

In [52]:
from langchain_core.output_parsers import StrOutputParser

In [53]:
parser = StrOutputParser()

In [54]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [55]:
from langchain_core.runnables import RunnablePassthrough

In [60]:
rag_chain = (
    {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
     | Prompt 
     | llm 
     | StrOutputParser()
)

In [61]:
rag_chain.invoke("tell me about llama2 finetuning benchmark experiment?")

"<think>\nOkay, I need to figure out how to answer the question about Llama 2 finetuning benchmark experiments based on the given context. Let me start by reading through the context carefully.\n\nThe context mentions that descriptions of the benchmarks and metrics can be found in Appendix A.4.7. It then talks about comparing Llama 2 to Llama 1-7B, specifically in the areas of hyper-specialization, Human-Eval, and MBPP code generation benchmarks. It also mentions evaluating Llama 2 on the NaturalQuestions and TriviaQA benchmarks, comparing it with other open-source models like Falcon and others, and Touvron et al., 2023.\n\nSo, the question is asking about the finetuning benchmark experiments for Llama 2. From the context, I see that Llama 2 was tuned on Human-Eval and MBPP code generation benchmarks. It was also evaluated on NaturalQuestions and TriviaQA. Comparisons were made with Llama 1 and Falcon.\n\nI should structure the answer by first stating that the context refers to Appendi

In [62]:
rag_chain.invoke("can you tell me about the Granular reward model accuracy per performance rating from table 8?")

'<think>\nOkay, so I need to figure out the answer to the question about the Granular reward model accuracy per performance rating from Table 8 based on the provided context. Let me go through this step by step.\n\nFirst, I\'ll read the question again to make sure I understand it correctly. The user is asking about the Granular reward model accuracy per performance rating from Table 8. So, I\'m looking for specific accuracy numbers related to performance ratings in Table 8.\n\nNow, I\'ll look at the context provided. The context includes some text about Tables 6, 7, and 8, along with some data. Let me break it down.\n\nThe context starts with some numbers under "Safety RM Meta Helpful. 64.6 57.5 53.8 52.2 56.2" and "Helpfulness RM 80.7 67.5 60.9 54.7 63.2". It mentions that Table 8 reports per-preference rating accuracy for the Granular reward model. So, these numbers are likely the accuracy figures for different performance ratings.\n\nNext, there are mentions of other tables: Table 6

In [63]:
rag_chain.invoke("can you tell me about scaling trends for the reward model?")

"<think>\nOkay, I need to figure out the scaling trends for the reward model based on the given context. Let me read through the context carefully. \n\nThe context mentions a figure, Figure 6, which shows scaling trends for the reward model. It says that more data and a larger model size generally improve performance. They studied this by tuning different model sizes using increasing amounts of data collected each week. \n\nThey also talk about using diverse open-source Reward Modeling datasets. Importantly, they haven't observed any divergence yet and think that iterative model updates might be preventing it. \n\nSo, putting this together, the scaling trends are positive—more data and bigger models lead to better results. They haven't hit any limits yet where adding more data or size doesn't help, which is a good sign. The iterative process might be a reason why they're not seeing the usual drop-offs that sometimes happen with scaling.\n</think>\n\nThe scaling trends for the reward mo